In [1]:
# check langchain version it should be 0.3 if not uncomment below pip command to install the correct version
import langchain, langchain_community
print(langchain.__version__)
print(langchain_community.__version__)
# !pip install langchain==0.3.27 langchain-openai==0.3.33 langchain-community==0.3.24

0.3.27
0.3.24


In [6]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.tools import tool

# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
# The LLM 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [8]:
@tool
def TrafficStatusTool(route: str) -> str:
    """Check current traffic conditions for a delivery route."""
    return f"Traffic on {route} is currently moderate. Approx delay: 10 minutes."

@tool
def VehicleCapacityTool(vehicle_id: str) -> str:
    """Check remaining load capacity of a specified delivery vehicle."""
    return f"Vehicle {vehicle_id} has 120 package capacity remaining."

@tool
def DeliveryWindowTool(address: str) -> str:
    """Check allowed customer delivery window for a specific address."""
    return f"Delivery window for {address}: 2PM–5PM."


tools = [TrafficStatusTool, VehicleCapacityTool, DeliveryWindowTool]

What We Have Now (Simulation Mode)

Right now, the tools return fixed, hardcoded responses.
That means no matter what route, vehicle, or address is passed, the response will always be the same.

What Would Change in a Real-Time Production System?

In a real system, instead of returning fixed text, each tool would:

| Tool                  | Current Behavior             | Real-time Behavior                                                                      |
| --------------------- | ---------------------------- | --------------------------------------------------------------------------------------- |
| `TrafficStatusTool`   | Returns a static delay       | Calls real mapping API (Google Maps etc) to get live congestion and ETA |
| `VehicleCapacityTool` | Returns fixed capacity       | Pulls live telematics/WMS/ERP data (FedEx-style truck load data)                        |
| `DeliveryWindowTool`  | Returns fake delivery window | Queries CRM/order system for actual customer SLA + priority rules                       |


In [11]:
# Deliberative Agent Prompt
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a logistics optimization AI assistant.
Your goal: plan the MOST efficient delivery route considering:
- Traffic patterns
- Delivery windows
- Vehicle capacity
- Time efficiency
Use a deliberate reasoning format:
Thought:
Action: <tool-name>
Action Input: <arguments>

Repeat until ready.
When fully confident, respond using this format:

Final Answer:
<optimized delivery plan>

User query:{input},                                ,
"placeholder", "{agent_scratchpad}
""")

In [ ]:
# Create Reasoning Agent
# ---------------------------------
from langchain.agents import AgentExecutor, create_tool_calling_agent
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

# 'verbose' shows step logs, and 'max_iterations' limits how many reasoning steps it can take.
executor = AgentExecutor(agent=agent, tools=tools, verbose=True,max_iterations=1)



In [ ]:
result = executor.invoke({
    "input": "Plan route for vehicle V-12 to deliver to 431 Lakeview Street and 92 Sunset Road"
})

print("\nOPTIMIZED PLAN")
print(result)



> Entering new AgentExecutor chain...
Thought: First, I need to check the delivery windows for both addresses to ensure that the deliveries can be made within the allowed time frames.

Action: functions.DeliveryWindowTool  
Action Input: { address: "431 Lakeview Street" }

Action: functions.DeliveryWindowTool  
Action Input: { address: "92 Sunset Road" }

> Finished chain.

📦 OPTIMIZED PLAN
{'input': 'Plan route for vehicle V-12 to deliver to 431 Lakeview Street and 92 Sunset Road', 'output': 'Thought: First, I need to check the delivery windows for both addresses to ensure that the deliveries can be made within the allowed time frames.\n\nAction: functions.DeliveryWindowTool  \nAction Input: { address: "431 Lakeview Street" }\n\nAction: functions.DeliveryWindowTool  \nAction Input: { address: "92 Sunset Road" }'}
